In [1]:
# BASIC IMPORTS
import sys
sys.path.append(r'..\src')

import pandas as pd

import pylab as plt

import seaborn as sns

import numpy as np

from src import getGraphic
from ufc_fights import set_names_upper, set_name_winner, fix_columns, set_nan_columns, add_fighterid
from ufc_fighters import load_ufc, set_fighter_upper, clean_fighters

In [2]:
# MACHINE LEARNING IMPORTS
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor as RFR
rfr=RFR()

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

import h2o
from h2o.automl import H2OAutoML

In [3]:
fighter = pd.read_csv(r'..\data\raw_fighter_details.csv')

data = pd.read_csv(r'..\data\data.csv')

ufc_fighters = pd.read_csv(r'..\data\ufc_fighters_official.csv')

# TABLA LUCHADORES

In [4]:
'''set_fighter_upper(fighter)

fighters = clean_fighters(fighter, ufc_fighters)

fighters'''

'set_fighter_upper(fighter)\n\nfighters = clean_fighters(fighter, ufc_fighters)\n\nfighters'

# TABLA FIGHTS

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Columns: 144 entries, R_fighter to R_age
dtypes: bool(1), float64(106), int64(28), object(9)
memory usage: 6.6+ MB


In [6]:
data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,0,1,0,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,0,3,0,0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,1,3,0,0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,0,0,0,0,Switch,160.02,160.02,115.0,28.0,25.0
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,1,2,2,0,Orthodox,167.64,172.72,135.0,29.0,43.0


In [7]:
# Las reglas oficiales de la UFC no entraron hasta el año 2001, así que solo cogemos combates por encima del año 2002
data = data[data.date>'2002-01-01']
# B_draw y R_draw son columnas con todos los valores 0
data = data.drop(['B_draw','R_draw'], axis=1)
# Eliminamos DRAW en Winner para ir probando
data = data[data.Winner!='Draw']

In [8]:
set_names_upper(data)

data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,ADRIAN YANEZ,GUSTAVO LOPEZ,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,0,1,0,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,TREVIN GILES,ROMAN DOLIDZE,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,0,3,0,0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,TAI TUIVASA,HARRY HUNSUCKER,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,1,3,0,0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,CHEYANNE BUYS,MONTSERRAT CONEJO,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,0,0,0,0,Switch,160.02,160.02,115.0,28.0,25.0
4,MARION RENEAU,MACY CHIASSON,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,1,2,2,0,Orthodox,167.64,172.72,135.0,29.0,43.0


In [9]:
'''set_name_winner(data)

data.head()'''

'set_name_winner(data)\n\ndata.head()'

In [10]:
fix_columns(data)

data

,R_fighter,B_fighter,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_SIG_STR_pct,B_avg_TD_pct,...,B_PCT_BODY,R_PCT_BODY,B_PCT_LEG,R_PCT_LEG,B_PCT_DISTANCE,R_PCT_DISTANCE,B_PCT_CLINCH,R_PCT_CLINCH,B_PCT_GROUND,R_PCT_GROUND
0,ADRIAN YANEZ,GUSTAVO LOPEZ,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.420000,0.330,...,48.0,50.0,40.0,NaN,26.0,33.0,30.0,50.0,40.0,NaN
1,TREVIN GILES,ROMAN DOLIDZE,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.660000,0.300,...,40.0,41.0,48.0,50.0,34.0,34.0,50.0,34.0,39.0,42.0
2,TAI TUIVASA,HARRY HUNSUCKER,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,NaN,44.0,NaN,45.0,NaN,31.0,NaN,43.0,NaN,43.0
3,CHEYANNE BUYS,MONTSERRAT CONEJO,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MARION RENEAU,MACY CHIASSON,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.535625,0.185,...,46.0,44.0,38.0,46.0,25.0,25.0,42.0,42.0,33.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721,RICCO RODRIGUEZ,JEFF MONSON,2002-01-11,"Uncasville, Connecticut, USA",Red,False,Heavyweight,0.000,0.400000,0.150,...,46.0,50.0,50.0,50.0,17.0,26.0,33.0,50.0,35.0,40.0
5722,MURILO BUSTAMANTE,DAVE MENNE,2002-01-11,"Uncasville, Connecticut, USA",Red,True,Middleweight,0.000,0.465000,0.500,...,46.0,50.0,43.0,50.0,23.0,34.0,39.0,38.0,37.0,32.0
5723,JENS PULVER,BJ PENN,2002-01-11,"Uncasville, Connecticut, USA",Red,True,Lightweight,0.750,0.772500,0.750,...,50.0,45.0,NaN,50.0,26.0,21.0,50.0,40.0,46.0,37.0
5724,KEVIN RANDLEMAN,RENATO SOBRAL,2002-01-11,"Uncasville, Connecticut, USA",Red,False,LightHeavyweight,0.000,0.410000,0.800,...,50.0,49.0,40.0,44.0,25.0,23.0,NaN,44.0,38.0,32.0


In [11]:
set_nan_columns(data)

data

,fight_id,R_fighter,B_fighter,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_SIG_STR_pct,...,B_PCT_BODY,R_PCT_BODY,B_PCT_LEG,R_PCT_LEG,B_PCT_DISTANCE,R_PCT_DISTANCE,B_PCT_CLINCH,R_PCT_CLINCH,B_PCT_GROUND,R_PCT_GROUND
0,0,ADRIAN YANEZ,GUSTAVO LOPEZ,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000000,0.420000,...,48.0,50.0,40.0,45.0,26.0,33.0,30.0,50.0,40.0,41.0
1,1,TREVIN GILES,ROMAN DOLIDZE,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500000,0.660000,...,40.0,41.0,48.0,50.0,34.0,34.0,50.0,34.0,39.0,42.0
2,2,TAI TUIVASA,HARRY HUNSUCKER,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,0.017578,0.449412,...,41.0,44.0,45.0,45.0,27.0,31.0,41.0,43.0,41.0,43.0
3,3,CHEYANNE BUYS,MONTSERRAT CONEJO,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,0.017578,0.449412,...,41.0,41.0,45.0,45.0,27.0,27.0,41.0,41.0,41.0,41.0
4,4,MARION RENEAU,MACY CHIASSON,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125000,0.535625,...,46.0,44.0,38.0,46.0,25.0,25.0,42.0,42.0,33.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721,5618,RICCO RODRIGUEZ,JEFF MONSON,2002-01-11,"Uncasville, Connecticut, USA",Red,False,Heavyweight,0.000000,0.400000,...,46.0,50.0,50.0,50.0,17.0,26.0,33.0,50.0,35.0,40.0
5722,5619,MURILO BUSTAMANTE,DAVE MENNE,2002-01-11,"Uncasville, Connecticut, USA",Red,True,Middleweight,0.000000,0.465000,...,46.0,50.0,43.0,50.0,23.0,34.0,39.0,38.0,37.0,32.0
5723,5620,JENS PULVER,BJ PENN,2002-01-11,"Uncasville, Connecticut, USA",Red,True,Lightweight,0.750000,0.772500,...,50.0,45.0,45.0,50.0,26.0,21.0,50.0,40.0,46.0,37.0
5724,5621,KEVIN RANDLEMAN,RENATO SOBRAL,2002-01-11,"Uncasville, Connecticut, USA",Red,False,LightHeavyweight,0.000000,0.410000,...,50.0,49.0,40.0,44.0,25.0,23.0,41.0,44.0,38.0,32.0


In [12]:
# ¿Borrar LOCATION?
# ¿Borrar filas donde weight_class es CATCHWEIGHT?

In [13]:
'''plt.figure(figsize=(50, 40))

corr_matrix = data.iloc[:,:].corr(method = 'pearson').abs()

sns.heatmap(corr_matrix, annot=True)'''

"plt.figure(figsize=(50, 40))\n\ncorr_matrix = data.iloc[:,:].corr(method = 'pearson').abs()\n\nsns.heatmap(corr_matrix, annot=True)"

In [14]:
'''correlations = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)).stack()
                    .sort_values(ascending=False))
print(correlations[:10])'''

'correlations = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)).stack()\n                    .sort_values(ascending=False))\nprint(correlations[:10])'

In [15]:
# FUNCTION FOR SELECT THE LAST OR FIRST FIGHT OF A CONCRETE FIGHTER

#  i = index of the fighter's fight, 0 means the last fight, -1 means first fight
def select_fight_row(df, name, i): 
    
    df_temp = df[(df['R_fighter'] == name) | (df['B_fighter'] == name)]  # filter df on fighter's name
    
    df_temp.reset_index(drop=True, inplace=True) #  as we created a new temporary dataframe, we have to reset indexes
    
    idx = max(df_temp.index)  #  get the index of the oldest fight
    
    if i > idx:  #  if we are looking for a fight that didn't exist, we return nothing
        
        return
    
    arr = df_temp.iloc[i,:].values
    
    return arr

select_fight_row(data, 'JON JONES', 0) #  we get the last fight of Amanda Nunes

array([528, 'JON JONES', 'DOMINICK REYES',
       Timestamp('2020-02-08 00:00:00'), 'Houston, Texas, USA', 'Red',
       True, 'LightHeavyweight', 0.71875, 0.599375, 0.03125, 0.03125, 0.0,
       0.84375, 0.03125, 19.40625, 407.6875, 10, 0, 6, 0, 0, 1, 1, 3, 1,
       0, 'Southpaw', 193.04, 195.58, 205.0, 0.0637826919555664,
       0.6548941135406494, 0.1523591423034668, 0.0171833038330078, 0.0,
       3.651911735534668, 1.0510683059692385, 224.74591541290283,
       1354.477560043335, 68, 14, 19, 2, 0, 1, 8, 5, 5, 0, 'Orthodox',
       193.04, 213.36, 205.0, 30.0, 32.0, 34.0, 42.0, 27.0, 32.0, 40.0,
       45.0, 48.0, 45.0, 30.0, 38.0, 46.0, 45.0, 43.0, 49.0], dtype=object)

In [16]:
# GET ALL ACTIVE FIGHTERS ACCORDING TO THE LIMIT DATE

def list_fighters(df, limit_date):
    
    df_temp = df[df['date'] > limit_date]
    
    set_R = set(df_temp['R_fighter'])
    set_B = set(df_temp['B_fighter'])
    
    fighters = list(set_R.union(set_B))
    
    return fighters

fighters = list_fighters(data, '2008-01-01')
len(fighters)

1787

In [17]:
# FUNCTION FOR BUILD A DATAFRAME WHICH HAS LAST/FIRST/... FIGHTS OF SELECTED FIGHTERS

def build_df(df, fighters, i):
    
    arr = [select_fight_row(df, fighters[f], i) for f in range(len(fighters)) if select_fight_row(df, fighters[f], i) is not None]
    
    cols = [col for col in df] 
    
    df_fights = pd.DataFrame(data=arr, columns=cols)
    
    df_fights.drop_duplicates(inplace=True)
    
    df_fights['title_bout'] = df_fights['title_bout'].replace({True: 1, False: 0})
    
    df_fights.drop(['R_fighter', 'B_fighter', 'date'], axis=1, inplace=True)
    
    return df_fights

df_train = build_df(data, fighters, 0)
df_test = build_df(data, fighters, 1)

In [18]:
df_train.shape, df_test.shape

((1534, 67), (1411, 67))

In [19]:
# AMOUNT OF WINNERS RED OR BLUE IN TRAIN AND TEST DATAFRAMES

print(len(df_train[df_train['Winner'] == 'Blue']))
print(len(df_train[df_train['Winner'] == 'Red']))

print(len(df_test[df_test['Winner'] == 'Blue']))
print(len(df_test[df_test['Winner'] == 'Red']))

602
932
520
891


In [20]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

preprocessor = make_column_transformer((OrdinalEncoder(), ['weight_class', 'B_Stance', 'R_Stance']), remainder='passthrough')

# If the winner is from the Red corner, Winner label will be encoded as 1, otherwise it will be 0 (Blue corner)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['Winner'])
y_test = label_encoder.transform(df_test['Winner'])

X_train, X_test = df_train.drop(['Winner'], axis=1), df_test.drop(['Winner'], axis=1)

In [21]:
'''# buscando colinealidad

plt.figure(figsize=(15, 10))

sns.set(style='white')

mask=np.triu(np.ones_like(data.corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)


sns.heatmap(data.corr(),
           mask=mask,
          cmap=cmap,
          center=0,
          square=True,
          linewidths=0.5,
          cbar_kws={'shrink': 0.5},
           annot=True);'''

"# buscando colinealidad\n\nplt.figure(figsize=(15, 10))\n\nsns.set(style='white')\n\nmask=np.triu(np.ones_like(data.corr(), dtype=bool))\n\ncmap=sns.diverging_palette(0, 10, as_cmap=True)\n\n\nsns.heatmap(data.corr(),\n           mask=mask,\n          cmap=cmap,\n          center=0,\n          square=True,\n          linewidths=0.5,\n          cbar_kws={'shrink': 0.5},\n           annot=True);"

### LOGISTIC REGRESSION AND RANDOM FOREST

In [22]:
'''# label encoding 

le=LabelEncoder()

for c in data.columns:
    
    if data.dtypes[c]==object:
        
        le.fit(data[c].astype(str))
        
        data[c]=le.transform(data[c].astype(str))

X = data.drop('Winner', axis=1)

y = data.Winner

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, train_size=0.8,
                                                    random_state=42)

logreg=LogisticRegression(max_iter=2000)

# RMSE POR MODELO
models = [logreg, rfr]

for m in models:
    
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    
    print(f'RMSE Model {m}: {mse(y_test, y_pred, squared=False)}')
    print(f'R2 Model {m}: {r2(y_test, y_pred)}')
    
sum(y_pred==y_test)/y_pred.shape[0]*100'''

"# label encoding \n\nle=LabelEncoder()\n\nfor c in data.columns:\n    \n    if data.dtypes[c]==object:\n        \n        le.fit(data[c].astype(str))\n        \n        data[c]=le.transform(data[c].astype(str))\n\nX = data.drop('Winner', axis=1)\n\ny = data.Winner\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, \n                                                    test_size=0.2, train_size=0.8,\n                                                    random_state=42)\n\nlogreg=LogisticRegression(max_iter=2000)\n\n# RMSE POR MODELO\nmodels = [logreg, rfr]\n\nfor m in models:\n    \n    m.fit(X_train, y_train)\n    y_pred = m.predict(X_test)\n    \n    print(f'RMSE Model {m}: {mse(y_test, y_pred, squared=False)}')\n    print(f'R2 Model {m}: {r2(y_test, y_pred)}')\n    \nsum(y_pred==y_test)/y_pred.shape[0]*100"

### H2O

In [23]:
'''train = data.iloc[:4580]

test = data.iloc[4580:].drop('Winner', axis=1)

train.shape, test.shape'''

"train = data.iloc[:4580]\n\ntest = data.iloc[4580:].drop('Winner', axis=1)\n\ntrain.shape, test.shape"

In [24]:
'''# H2O

h2o.init()

# parsear datos para h20

h2train=h2o.H2OFrame(train)

h2test=h2o.H2OFrame(test)

X=[c for c in h2train.columns if c!='Winner']

y='Winner'

# inicia auto-machine-learning

automl=H2OAutoML(max_models=50,
                 seed=42,
                 max_runtime_secs=300,
                 sort_metric='RMSE')'''

"# H2O\n\nh2o.init()\n\n# parsear datos para h20\n\nh2train=h2o.H2OFrame(train)\n\nh2test=h2o.H2OFrame(test)\n\nX=[c for c in h2train.columns if c!='Winner']\n\ny='Winner'\n\n# inicia auto-machine-learning\n\nautoml=H2OAutoML(max_models=50,\n                 seed=42,\n                 max_runtime_secs=300,\n                 sort_metric='RMSE')"

In [25]:
'''# entrena

automl.train(x=X,
             y=y,
             training_frame=h2train)'''

'# entrena\n\nautoml.train(x=X,\n             y=y,\n             training_frame=h2train)'

In [26]:
'''# prediciones del lider

y_pred = automl.leader.predict(h2test)

y_pred = y_pred.as_data_frame()

y_pred'''

'# prediciones del lider\n\ny_pred = automl.leader.predict(h2test)\n\ny_pred = y_pred.as_data_frame()\n\ny_pred'